In [40]:
# re-import modules without restarting kernel
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


from io_helpers import *
from matching_helpers import *

#essential imports
import pandas as pd
import numpy as np

#date
import datetime as dt

from IPython.display import clear_output



cps = 7



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Matching movies and dead actors/directors

### 1) Load and clean the meta_movies data generated with Amazon API

In [2]:
## open datafiles
##

features_meta = ['asin', 'actors', 'directors', 'creators', 'authors']

MMTV = pd.read_csv('DATA/metadata_processed/meta_Movies_and_TV_processed(v1).csv', low_memory=False)[features_meta]
AIV = pd.read_csv('DATA/metadata_processed/meta_Amazon_Instant_Video_processed(v1).csv', low_memory=False)[features_meta]

video_df = pd.concat((MMTV, AIV))
video_df = df_rm_punctuation(video_df)

In [3]:
assert(MMTV.shape[0]+AIV.shape[0] == video_df.shape[0])
video_df.head()

,asin,actors,directors,creators,authors
0,0000143561,,,,
1,0000589012,,,,Joe Herzanek
2,0000695009,,,,Epilepsy Foundation
3,000107461X,Douglas Miller,,"(Joe Thomas,Producer),(Joseph Williams,Producer)",
4,0000143529,Alton Brown,,,


## 2) Clean and filter dead actors data

In [4]:
deaths = pd.read_csv('DATA/deaths.csv', low_memory=False, encoding="ISO-8859-1")
dead_actors = deaths[deaths['Actor']==True].drop('Actor', axis=1).drop('Author', axis=1).drop('Musician', axis=1).drop('Description', axis=1)
# for some reason the index skips some numbers
dead_actors = dead_actors.reset_index().drop('index', axis=1)
dead_actors['Name'] = std_actors_names(dead_actors['Name'])
dead_actors.head()

,Name,Birth Date,Death Date
0,jack weston,1924,1996-05-03
1,john beradino,1917,1996-05-19
2,jon pertwee,1919,1996-05-20
3,lash larue,1917,1996-05-21
4,enrique álvarez félix,1934,1996-05-24


## 3) Build (metadata<->actors) controls list

In [5]:
# filter out blanks, products_actors will be re-used
amazon_actor_products = video_df[(video_df["actors"].isnull()==False)]
amazon_actor_products = amazon_actor_products[amazon_actor_products['actors']!='']
print(amazon_actor_products.shape)
amazon_actor_products.head()

(144720, 5)


,asin,actors,directors,creators,authors
3,000107461X,Douglas Miller,,"(Joe Thomas,Producer),(Joseph Williams,Producer)",
4,0000143529,Alton Brown,,,
5,0000143502,Alton Brown,,,
6,0000143588,Ina Garten,,,
11,0001485423,Sr Faustina,J Paddy Nolan,,


In [6]:
amazon_actor_names = [troup.split(',') for troup in list(amazon_actor_products['actors'])]
amazon_actor_names = [lm.lower() for lm in np.concatenate(amazon_actor_names)]
amazon_actor_names = list(set(amazon_actor_names))

# weird... 
print(amazon_actor_names[:5])
#amazon_actor_names = amazon_actor_names[:1]+ amazon_actor_names[2:] 
amazon_actor_names = amazon_actor_names[1:]

amazon_actors = pd.DataFrame(amazon_actor_names, columns=['Name'])
amazon_actors['Name'] = std_actors_names(amazon_actors['Name'])
amazon_actors['Birth Date'] = pd.Series()
amazon_actors['Death Date'] = pd.Series()
amazon_actors.head()

['angela cardile', '', 'luciana pedraza', 'rosanna desoto', 'dennis falt']


,Name,Birth Date,Death Date
0,,NaN,NaN
1,luciana pedraza,NaN,NaN
2,rosanna desoto,NaN,NaN
3,dennis falt,NaN,NaN
4,louis mouchet,NaN,NaN


In [7]:
## filter out deads
##

dead_actors['clname'] = dead_actors['Name'].map(cleanstr)
dead_actors = dead_actors.drop('Name', axis=1)
nondead_actors = pd.DataFrame(list(map(lambda s : cleanstr(s), amazon_actor_names)),columns=['clname'])
nondead_actors= nondead_actors[nondead_actors['clname'].map(lambda e : len(e.split())>1)]

for idx, name in enumerate(dead_actors['clname'].tolist()):
    if idx % 100 == 0 : 
        print(idx)
    mapp = nondead_actors['clname'].str.contains(name)
    if mapp.any():
        nondead_actors = nondead_actors[mapp==False] 

nondead_actors['Birth Date'] = pd.Series()
nondead_actors['Death Date'] = pd.Series()
nondead_actors.head()

controls = nondead_actors

all_actors = pd.concat((dead_actors, nondead_actors))

0
100
200
300
400
500
600


In [8]:
controls.shape, dead_actors.shape, amazon_actors.shape, amazon_actors.shape[0]-controls.shape[0]

((149407, 3), (639, 3), (155561, 3), 6154)

## 4) Match dead actors and meta data

In [9]:
pd.options.mode.chained_assignment = None 
meta_dead_actors = pd.DataFrame()

amazon_actor_products['clname'] = amazon_actor_products['actors'].map(cleanstr)

for idx, actor in enumerate(list(dead_actors['clname'])):
    
    match = amazon_actor_products[amazon_actor_products['clname'].str.contains(actor)==True].drop('actors',axis=1)
    match['clname'] = actor
    match['Death Date'] = dead_actors.reset_index()['Death Date'].get_value(idx)
    meta_dead_actors = meta_dead_actors.append(match)
    
    #if match.empty: print(actor)
    
    if idx%100==0 and idx!=0: 
        print('actor {} in {} - Done'.format(idx, len(list(dead_actors['clname']))))
        
print(meta_dead_actors.shape)        
meta_dead_actors.head()

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


actor 100 in 639 - Done
actor 200 in 639 - Done
actor 300 in 639 - Done
actor 400 in 639 - Done
actor 500 in 639 - Done
actor 600 in 639 - Done
(14813, 6)


,asin,directors,creators,authors,clname,Death Date
3487,0792840461,Norman Jewison,"(Haskell Wexler,Cinematographer),(Norman Jewis...",,jack weston,1996-05-03
10823,6300216217,,,,jack weston,1996-05-03
11212,6300270297,Richard Lester,"(Paul Wilson,Cinematographer),(John Bloom,Edit...",,jack weston,1996-05-03
11233,6300270025,Brian G Hutton,"(Andre Morgan,Producer),(Daniel Grodnik,Produc...",,jack weston,1996-05-03
12506,6301514017,Stuart Rosenberg,,,jack weston,1996-05-03


In [10]:
amazon_actor_products[amazon_actor_products['clname'].str.contains('william hickey ')]

,asin,actors,directors,creators,authors,clname


In [11]:
len(set(meta_dead_actors['clname']))

501

## 5) Subsample control actors and match meta data

In [12]:
from numpy.random import randint

def subsample_df (df, nbWanted) : 
    assert nbWanted < len(df)
    randomIndex = randint(0, len(df)-1, nbWanted)
    return df.loc[randomIndex]

sub_controls = subsample_df(controls, nb_controls_per_subject*len(dead_actors))
print(sub_controls.shape)
sub_controls.head(3)

(2556, 3)


/usr/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,clname,Birth Date,Death Date
146180,michael hynson,NaN,NaN
148209,johnny vidacovich,NaN,NaN
147887,NaN,NaN,NaN


In [13]:
pd.options.mode.chained_assignment = None 
meta_control_actors = pd.DataFrame()

d=''
for idx, actor in enumerate(list(sub_controls.dropna(how='all')['clname'])):  
    match = pd.DataFrame()
    match = amazon_actor_products[amazon_actor_products['clname'].str.contains(actor)==True].drop('actors',axis=1)
    match['clname'] = actor
    match['Death Date'] = sub_controls.reset_index()['Death Date'].get_value(idx)
    meta_control_actors = meta_control_actors.append(match)
    
    if idx%100==0 and idx!=0: 
        print('actor {} in {} - Done'.format(idx, len(list(sub_controls['clname']))))


print(meta_control_actors.shape)
print(len(set(meta_dead_actors['clname'])))
meta_control_actors.head()

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:9: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


actor 100 in 2556 - Done
actor 200 in 2556 - Done
actor 300 in 2556 - Done
actor 400 in 2556 - Done
actor 500 in 2556 - Done
actor 600 in 2556 - Done
actor 700 in 2556 - Done
actor 800 in 2556 - Done
actor 900 in 2556 - Done
actor 1000 in 2556 - Done
actor 1100 in 2556 - Done
actor 1200 in 2556 - Done
actor 1300 in 2556 - Done
actor 1400 in 2556 - Done
actor 1500 in 2556 - Done
actor 1600 in 2556 - Done
actor 1700 in 2556 - Done
actor 1800 in 2556 - Done
actor 1900 in 2556 - Done
actor 2000 in 2556 - Done
actor 2100 in 2556 - Done
actor 2200 in 2556 - Done
actor 2300 in 2556 - Done
actor 2400 in 2556 - Done
(7822, 6)
501


,asin,directors,creators,authors,clname,Death Date
21627,630327417X,Bruce Brown,"(Bruce Brown,Cinematographer),(Bruce Brown,Edi...",,michael hynson,NaN
64599,B0001XALWI,,,,johnny vidacovich,NaN
147557,B0023AV874,Will Richardson,,,greg bowdish,NaN
147311,B0020TS5D8,"Fred Calvert,David J Negron Jr",,,joseph gatt,NaN
14739,B007UXSKZ2,Mary Lambert,"(Mary Lambert,Writer),(Mark Liever;Elizabeth S...",,joseph gatt,NaN


## 6) Concatenate (metadatas<->artist) for dead and control artists

In [14]:
meta_all_actors = pd.concat((meta_dead_actors, meta_control_actors))

## 7) Loading Reviews data for Movies and TV and cleaning

In [15]:
f_MTV = 'DATA/review/reviews_Movies_and_TV.json.gz'
f_AIV = 'DATA/review/reviews_Amazon_Instant_Video.json.gz'
#features_reviews = ['asin', 'reviewTime']
features_reviews = ['asin', 'reviewTime', 'overall', 'helpful', 'summary', 'reviewText']

In [16]:
b=gz_to_dataframe(f_AIV)[features_reviews]
a=gz_to_dataframe(f_MTV)[features_reviews]
safe_reviews_df = pd.concat((a, b))

In [17]:
reviews_df = safe_reviews_df
print(reviews_df.shape)
reviews_df.head()

(5190980, 6)


,asin,reviewTime,overall,helpful,summary,reviewText
0,0000143502,"01 17, 2013",5.0,"[0, 0]",Alton... nough said,This has some great tips as always and is help...
1,0000143529,"10 2, 2013",5.0,"[0, 0]",Ah Alton...,This is a great pastry guide. I love how Alto...
2,0000143561,"07 17, 2008",2.0,"[2, 4]",Don't waste your money,I have to admit that I am a fan of Giada's coo...
3,0000143588,"03 13, 2009",5.0,"[9, 9]",VOLUME 1 & VOLUME 2-BETTER THAN THERAPY,I bought these two volumes new and spent over ...
4,0000143588,"01 18, 2009",5.0,"[1, 4]",Barefoot Contesst Vol 2,I am very pleased with the dvd only wish i cou...


In [18]:
def prefilter_reviews(review_df):
    filtered_reviews = review_df[review_df['asin'].isin(meta_all_actors['asin'].tolist())]
    filtered_reviews = filtered_reviews[features_reviews]
    filtered_reviews['reviewTime'] = list(pd.to_datetime(filtered_reviews['reviewTime'].str.replace(',','').str.replace(' ','-')))
    
    return filtered_reviews

In [19]:
filtered_reviews = prefilter_reviews(reviews_df)
filtered_reviews = filtered_reviews.reset_index()
print(filtered_reviews.shape)
filtered_reviews.head()

(744481, 7)


,index,asin,reviewTime,overall,helpful,summary,reviewText
0,720,0307141071,2000-08-03,5.0,"[12, 12]",Sweet Story,What a wonderful story. My daughter watches M...
1,721,0307141071,2005-02-15,5.0,"[0, 0]",A must-see for all ages!,This is my most favored Madeline episode. And ...
2,722,0307141128,2001-12-17,5.0,"[3, 3]",A favorite in our house,My daughter would watch this everyday if she c...
3,746,0307141985,2010-11-08,3.0,"[2, 7]",the not so great olden days of cinema...,"I purchased this video for my daughter, rememb..."
4,747,0307141985,2008-11-25,5.0,"[2, 2]",Nostalgic Entertainment,I video taped this off of television for my ch...


## 8) Matching meta/dead data with review data

In [20]:
print(meta_dead_actors.shape)
meta_dead_actors.head()

(14813, 6)


,asin,directors,creators,authors,clname,Death Date
3487,0792840461,Norman Jewison,"(Haskell Wexler,Cinematographer),(Norman Jewis...",,jack weston,1996-05-03
10823,6300216217,,,,jack weston,1996-05-03
11212,6300270297,Richard Lester,"(Paul Wilson,Cinematographer),(John Bloom,Edit...",,jack weston,1996-05-03
11233,6300270025,Brian G Hutton,"(Andre Morgan,Producer),(Daniel Grodnik,Produc...",,jack weston,1996-05-03
12506,6301514017,Stuart Rosenberg,,,jack weston,1996-05-03


In [21]:
pd.options.mode.chained_assignment = None  # default='warn'

def matching_meta_reviews(filtered_reviews, meta):
    
    processed_reviews = pd.DataFrame()
    
    for idx, asin in enumerate(meta['asin']):
        match_reviews = pd.DataFrame()
        match_reviews = filtered_reviews[filtered_reviews['asin']==asin]

        match_reviews['actor name'] = meta.reset_index()['clname'].get_value(idx)
        match_reviews['death date'] = meta.reset_index()['Death Date'].get_value(idx)

        processed_reviews = processed_reviews.append(match_reviews)

        if idx %1000 == 0 and idx!=0: 
            print('{} in {}'.format(idx, len(meta['asin'])))
        
    return processed_reviews

In [22]:
deads_reviews = matching_meta_reviews(filtered_reviews, meta_dead_actors)
print(deads_reviews.shape)
deads_reviews.head()

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


1000 in 14813
2000 in 14813
3000 in 14813
4000 in 14813
5000 in 14813
6000 in 14813
7000 in 14813
8000 in 14813
9000 in 14813
10000 in 14813
11000 in 14813
12000 in 14813
13000 in 14813
14000 in 14813
(597276, 9)


,index,asin,reviewTime,overall,helpful,summary,reviewText,actor name,death date
95792,308659,0792840461,2006-04-07,4.0,"[1, 3]",The orginal,The Original and the best-not much to say read...,jack weston,1996-05-03
95793,308660,0792840461,2013-03-24,5.0,"[0, 0]",Steve McQueen invented cool,This was a great movie Steve McQueen invented ...,jack weston,1996-05-03
95794,308661,0792840461,2014-04-05,4.0,"[1, 1]",Where The Dough Is,"Everybody knows banks, whether in storefronts,...",jack weston,1996-05-03
95795,308662,0792840461,2012-12-21,2.0,"[3, 3]","great movie,but PLEASE avoid this copy.",an all-time great movie and one of my favorite...,jack weston,1996-05-03
95796,308663,0792840461,2013-05-12,5.0,"[0, 0]",It's McQueen,Fans of a rough and tough McQueen know that th...,jack weston,1996-05-03


## 8) Matching controls and reviews

In [23]:
print(meta_control_actors.shape)
meta_control_actors.sort_values(by='asin').head()

(7822, 6)


,asin,directors,creators,authors,clname,Death Date
82,0307141071,,,,andrea libman,NaN
101,0307141128,,,,andrea libman,NaN
79,0307142426,,,,andrea libman,NaN
86,0307142620,,,,andrea libman,NaN
94,0307142698,,,,andrea libman,NaN


In [24]:
def match_control_meta_reviews(filtered_reviews, meta):
    
    processed_reviews = pd.DataFrame()
    
    for idx, asin in enumerate(meta['asin']):
        match_reviews = pd.DataFrame()
        match_reviews = filtered_reviews[filtered_reviews['asin']==asin]

        match_reviews['actor name'] = meta.reset_index()['clname'].get_value(idx)

        processed_reviews = processed_reviews.append(match_reviews)

        if idx %1000 == 0 and idx!=0: 
            print('{} in {}'.format(idx, len(meta['asin'])))
        
    return processed_reviews

In [25]:
ctrls_reviews = match_control_meta_reviews(filtered_reviews, meta_control_actors)

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:9: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


1000 in 7822
2000 in 7822
3000 in 7822
4000 in 7822
5000 in 7822
6000 in 7822
7000 in 7822


## Save files

In [41]:
deads_reviews.to_csv('DATA/review_filtered/movie_reviews_matched_'+str(cps)+'cps.csv')

In [42]:
ctrls_reviews.to_csv('DATA/review_filtered/movie_reviews_matched_'+str(cps)+'_ctrls.csv')

## problems ?

In [28]:
deads_reviews.groupby('actor name').count().sort_values(by='index', ascending=False)[:10]

,index,asin,reviewTime,overall,helpful,summary,reviewText,death date
actor name,,,,,,,,
james stewart,10103,10103,10103,10103,10103,10103,10103,10103
charlton heston,9428,9428,9428,9428,9428,9428,9428,9428
james garner,7547,7547,7547,7547,7547,7547,7547,7547
marlon brando,7466,7466,7466,7466,7466,7466,7466,7466
paul newman,7285,7285,7285,7285,7285,7285,7285,7285
philip seymour hoffman,6470,6470,6470,6470,6470,6470,6470,6470
deforest kelley,6466,6466,6466,6466,6466,6466,6466,6466
mickey rooney,6295,6295,6295,6295,6295,6295,6295,6295
gregory peck,6187,6187,6187,6187,6187,6187,6187,6187


In [29]:
ctrls_reviews.groupby('actor name').count().sort_values(by='index', ascending=False)

,index,asin,reviewTime,overall,helpful,summary,reviewText
actor name,,,,,,,
matt damon,19321,19321,19321,19321,19321,19321,19321
laurence fishburne,9970,9970,9970,9970,9970,9970,9970
emily watson,7449,7449,7449,7449,7449,7449,7449
steve martin,6988,6988,6988,6988,6988,6988,6988
michelle williams,6724,6724,6724,6724,6724,6724,6724
liev schreiber,6259,6259,6259,6259,6259,6259,6259
ian anderson,5658,5658,5658,5658,5658,5658,5658
gérard depardieu,5356,5356,5356,5356,5356,5356,5356
tyler perry,5281,5281,5281,5281,5281,5281,5281


In [30]:
### PROBLEM
###

#ctrls_reviews[ctrls_reviews['actor name']=='johnny depp'].groupby('asin').count()['asins']
idx = meta_control_actors[meta_control_actors['clname']=='chris fisher'].groupby('asin').count().reset_index()['asin']
reviews_df[reviews_df['asin'].isin(idx)].count()

asin          0
reviewTime    0
overall       0
helpful       0
summary       0
reviewText    0
dtype: int64

In [31]:
match_reviews = filtered_reviews[filtered_reviews['asin'].str.contains('B00G3NK8NW')]
match_reviews.shape

(0, 7)

In [32]:
meta_all_actors[meta_all_actors['asin'].isin(['B00G3NK8NW'])]

,asin,directors,creators,authors,clname,Death Date


In [33]:
reviews_df[reviews_df['asin'].isin(['B00G3NK8NW'])].shape

(12, 6)

In [34]:
filtered_reviews[filtered_reviews['asin'].isin(['B00G3NK8NW'])].shape

(0, 7)

In [35]:
len(list(meta_all_actors['asin']))

22635

In [36]:
len(meta_all_actors['asin'].unique()), len(meta_all_actors['asin'])

(19511, 22635)

In [37]:
ctrls_reviews[ctrls_reviews['asin'].isin(['B00G3NK8NW'])]

,index,asin,reviewTime,overall,helpful,summary,reviewText,actor name
